In [1]:
import duckdb

In [2]:
con = duckdb.connect('../localdatabase.db')
con.sql("INSTALL httpfs; LOAD httpfs;")
con.sql("INSTALL ducklake; LOAD ducklake;")

In [3]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'marketing-and-ecommerce',
    'endpoint': 'localhost:9000'
}

# --- 1. Konfigurasi MinIO (S3 API) ---
con.sql(f"SET s3_endpoint = '{minioAuth['endpoint']}';") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_use_ssl = false;")
con.sql("SET s3_url_style = 'path';")

In [4]:
dbPath = '../localstagingdatabase.db'

In [5]:
con.sql(f"ATTACH 'ducklake:{dbPath}' AS datalakehouse;")
con.sql("USE datalakehouse;")

In [7]:
con.sql("SELECT * FROM ducklake_snapshots('datalakehouse')")

┌─────────────┬───────────────────────────────┬────────────────┬─────────────────────────────────────────────────────────────────────────────────────────┬─────────┬────────────────┬───────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                                         changes                                         │ author  │ commit_message │ commit_extra_info │
│    int64    │   timestamp with time zone    │     int64      │                                 map(varchar, varchar[])                                 │ varchar │    varchar     │      varchar      │
├─────────────┼───────────────────────────────┼────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┼─────────┼────────────────┼───────────────────┤
│           0 │ 2025-12-14 13:36:42.812363+07 │              0 │ {schemas_created=[main]}                                                                │ NULL    │ NULL           │ NULL      

In [ ]:
con.close()